In [4]:
%reset -s -f
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys 
sys.path.append("./../")

In [73]:
import pandas as pd
toydata = dict(
    cola=[1,1,1,1,3,2,3,5],
    colb=[3,1,3,1,2,2,4,4],
    colc=[3,1,3,3,2,2,4,4],
    target=["a","b","a","b","c","a","c","b"]
)
toy_df = pd.DataFrame(toydata)


In [ ]:


col = "colb"
d = toy_df.groupby(['target',col]).size().reset_index(name="Size")
d_error = d[d.duplicated(subset=[col],keep=False)]
print(d_error)
print(d_error.groupby(['target']).apply(lambda x: 0 if x.shape[0] == 1 else min(x["Size"]) / sum(x["Size"])))
# .mean())
d_correct = d[~d.duplicated(subset=[col],keep=False)]
d_correct.groupby(['target']).apply(lambda x : {col : x[col].tolist()})
# 

In [91]:
import numpy as np
from collections import Counter
from copy import deepcopy
from src.metric import get_weighted_gini_impurity , make_gini_attributes , get_min_giniattributes



for col in ["cola","colb","colc"] :
    print(col, get_weighted_gini_impurity(df=toy_df , col=col,target_col="target"))
result = make_gini_attributes(toy_df , cols =["cola","colb","colc"],target_col="target")
get_min_giniattributes(result)

cola 0.25
colb 0.25
colc 0.41666666666666663


('cola', 0.25)

In [7]:
col = "cola"

def get_metric_by_col(df , target, col) :
    count_table = df.groupby([target,col]).size().reset_index(name="Size")
    metric = count_table[count_table.duplicated(subset=[col],keep=False)].\
        groupby([col]).apply(lambda x : min(x["Size"]) / sum(x["Size"])).\
            mean()
    return metric 
get_metric_by_col(toy_df,"target",col=col)
count_table = toy_df.groupby(["target",col]).size().reset_index(name="Size")
print(count_table[~count_table.duplicated(subset=[col],keep=False)])
print(count_table)
# 1,2,5 / 3 

  target  cola  Size
1      a     2     1
3      b     5     1
4      c     3     2
  target  cola  Size
0      a     1     2
1      a     2     1
2      b     1     2
3      b     5     1
4      c     3     2


In [72]:
count_table = toy_df.groupby(["cola","target"]).size().reset_index(name='Size')
metric = count_table.groupby(['cola']).apply(lambda x : min(x["Size"]) / sum(x["Size"])).mean()


def get_error_table(count_table , target) :
    return count_table.groupby([target]).apply(lambda x : 0 if x.shape[0] == 1  else min(x["Size"]) / sum(x["Size"])  )

def get_error_rate(count_table , target) :

    metric = get_error_table(count_table , target).mean()
    return metric 

metric_cola = get_error_rate(count_table,target="target")
metric_cola

0.0

In [125]:
toy_df.groupby(["colb","target"]).size()
count_table = toy_df.groupby(["colb","target"]).size().reset_index(name='Size')

def make_count_table(df , target , by) :
    count_table = df.groupby([by,target]).size().reset_index(name='Size')
    return count_table 

def get_metric(df , target ,col) :
    count_table = make_count_table(df , target , col )
    metric = get_error_rate(count_table,target=col)
    return metric  



count_table = make_count_table(toy_df , "target","colc")
metric_colb = get_error_rate(count_table,target="colc")
metric_colb
count_table

,colc,target,Size
0,1,b,1
1,2,c,1
2,2,d,1
3,3,a,2
4,3,b,1
5,4,c,1
6,4,d,1


In [126]:
metric_result = {}
for i in ["cola","colb","colc"] :
    metric_result[i] = get_metric(toy_df , "target",i)
else :
    print(metric_result)

{'cola': 0.16666666666666666, 'colb': 0.25, 'colc': 0.3333333333333333}


In [127]:
def get_min_col(metric_result) :
    return min(metric_result.items(), key=lambda x: x[1]) [0]
get_min_col(metric_result)


'cola'

In [134]:

count_table = make_count_table(toy_df , "target","cola")
result = get_error_table(count_table=count_table,target="cola")
finish_pattern = result[result==0].index.tolist()
print("finish", "cola", finish_pattern)
non_finish_pattern = result[result!=0].index.tolist()
print("non finish", "cola", non_finish_pattern)
toy_df2 = toy_df[toy_df.cola.isin(non_finish_pattern)]

metric_result = {}
for i in ["colb","colc"] :
    metric_result[i] = get_metric(toy_df2 , "target",i)
else :
    print(metric_result)

finish cola [2, 3]
non finish cola [1]
{'colb': 0.0, 'colc': 0.16666666666666666}


In [130]:

count_table = make_count_table(toy_df2 , target="target",by="colb")
result = get_error_table(count_table=count_table,target="colb")
result

colb
1    0
3    0
dtype: int64

In [132]:
finish_pattern = result[result==0].index.tolist()
non_finish_pattern = result[result!=0].index.tolist()
toy_df3 = toy_df2[toy_df2.colb.isin(non_finish_pattern)]
toy_df3

,cola,colb,colc,target


In [115]:
toy_df2.groupby(['colb','target']).size().reset_index(name='Size')

,colb,target,Size
0,1,b,2
1,3,a,2


In [ ]:

toy_df2

In [ ]:

count_table = make_count_table(toy_df2 , "target","colb")
get_error_table(count_table=count_table,target="colb")
toy_df2

In [102]:
count_table

,colb,target,Size
0,1,b,2
1,2,c,1
2,2,d,1
3,3,a,2
4,4,c,1
5,4,d,1


In [98]:
non_finish_pattern

[2, 4]

In [99]:
toy_df2.colb.isin(non_finish_pattern)

0    False
1    False
2    False
3    False
Name: colb, dtype: bool